# Example FID
for the Ph.D. Training program of the German Chapter of the ISMRM

https://github.com/pulseq/MR-Physics-with-Pulseq/tree/main/tutorials/01_basic_sequences/notebooks

In [ ]:
!pip show pypulseq || pip install install git+https://github.com/imr-framework/pypulseq.git
!wget https://raw.githubusercontent.com/pulseq/MR-Physics-with-Pulseq/main/utils/utils.py

In [ ]:
import math
import warnings
import numpy as np
from matplotlib import pyplot as plt
import pypulseq as mr
import utils # several helper functions for simulation and recon

In [ ]:
# define high level parameters

fov=256e-3
dwell=10e-5
Nread=64
Nphase=1
TE=5e-3
TR=60e-3

# SETUP
# Set system limits
system = mr.Opts(
    max_grad=32,
    grad_unit="mT/m",
    max_slew=130,
    slew_unit="T/m/s",
    rf_ringdown_time=30e-6,
    rf_dead_time=100e-6,
)

# Create 90 degree slice selection pulse and gradient
rf_ex, gz1, _ = mr.make_sinc_pulse(
    flip_angle=90 * np.pi / 180,
    system=system,
    duration=1e-3,
    slice_thickness=5e-3,
    apodization=0.5,
    time_bw_product=4,
    phase_offset=0* np.pi / 180,
    return_gz=True,
)


print(f"""The bandwidth is {1/dwell:.2f} Hz
and {1/(dwell*Nread):.2f} Hz/px for the {Nread*dwell*1000:.2f} ms ADC.""")

adc = mr.make_adc(
    num_samples=Nread,
    duration=Nread*dwell,
    phase_offset=0* np.pi / 180,
    system=system,)


In [10]:
# CONSTRUCT SEQUENCE
# Create a new sequence object
seq = mr.Sequence(system)
seq.add_block(rf_ex)
seq.add_block(adc)

In [ ]:
# check sequence timing
(ok, error_report) = seq.check_timing()  # Check whether the timing of the sequence is correct
if ok:
    print("Timing check passed successfully")
else:
    print("Timing check failed. Error listing follows:")
    [print(e) for e in error_report]

#Visualization and Export

In [ ]:
#@title plot sequence
# plot the entire sequence
seq.plot()


# plot just a small initial part
print('zoom in')
seq.plot(time_range=[0, 0.002], time_disp="ms") #


In [ ]:
#@title write seq file and export
seq_filename='FID.seq'
seq.write(seq_filename)
from google.colab import files
files.download(seq_filename)  # Download locally

#Simulation

In [ ]:
#@title simulation setup
!pip install MRzeroCore &> /dev/null
!wget https://github.com/MRsources/MRzero-Core/raw/main/documentation/playground_mr0/numerical_brain_cropped.mat &> /dev/null
import MRzeroCore as mr0
import utils # several helper functions for simulation and recon

In [ ]:
#@title simulation (simple)
# kscape_adc.shape is [N_coils, N_meas, N_adc]
kspace_adc=utils.simulate_2d(seq, noise_level=0, n_coils=1, dB0=+10, B0_scale=1, B0_polynomial=None)
#sp_adc, t_adc = util.pulseq_plot(seq,signal=kspace_adc) # for pypulseq below dev branch.
seq.plot(plot_now=False)
mr0.util.insert_signal_plot(seq, kspace_adc)
plt.show()


In [ ]:
#@title simulation (advanced)
# %% S4: SETUP SPIN SYSTEM/object on which we can run the seq seq_filename
sz = [64, 64]

# (i) load a phantom object from file
obj_p = mr0.VoxelGridPhantom.load_mat('numerical_brain_cropped.mat')
obj_p = obj_p.interpolate(sz[0], sz[1], 1)
# Manipulate loaded data
obj_p.T2dash[:] = 5e-3
obj_p.D *= 0
obj_p.B0 = obj_p.B0*0+1000    # alter the B0 inhomogeneity
# Store PD for comparison
PD = obj_p.PD
B0 = obj_p.B0

obj_p.plot()
# Convert Phantom into simulation data
obj_p = obj_p.build()


# %% S5:. SIMULATE  the external.seq file and add acquired signal to ADC plot

# Read in the sequence
seq0 = mr0.Sequence.import_file(seq_filename)
seq0.plot_kspace_trajectory()
# Simulate the sequence
graph = mr0.compute_graph(seq0, obj_p, max_state_count=1000, min_state_mag=1e-5)
signal = mr0.execute_graph(graph, seq0, obj_p)

# PLOT sequence with signal in the ADC subplot
#sp_adc, t_adc = util.pulseq_plot(seq,signal=kspace_adc) # for pypulseq below dev branch.
seq.plot(plot_now=False)
mr0.util.insert_signal_plot(seq, signal)
plt.show()